# Uber project
## Predicting the hottest pick-up zones on Manhattan

### Install & Import the required packages

In [ ]:
%pip install -r requirements.txt

In [1]:
# For statistics and pre-processing
import pandas as pd
import time

# For Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# For visualizations
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Loading the dataset

In [2]:
# This is the main file
# Base : The TLC base company code affiliated with the Uber pickup (source: Kaggle)
data = pd.read_csv('./src/uber-raw-data-apr14.csv')
data.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [3]:
# FYI = This is the list of Location ID used in the main dataset
data_loc = pd.read_csv('./src/taxi-zone-lookup.csv')
data_loc.head(10)

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights
5,6,Staten Island,Arrochar/Fort Wadsworth
6,7,Queens,Astoria
7,8,Queens,Astoria Park
8,9,Queens,Auburndale
9,10,Queens,Baisley Park


### Basic informations on the dataset

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564516 entries, 0 to 564515
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Date/Time  564516 non-null  object 
 1   Lat        564516 non-null  float64
 2   Lon        564516 non-null  float64
 3   Base       564516 non-null  object 
dtypes: float64(2), object(2)
memory usage: 17.2+ MB


In [5]:
# No missing value in the dataset
data.isna().sum()

Date/Time    0
Lat          0
Lon          0
Base         0
dtype: int64

In [6]:
# There were close to 565k pickups recorded on Manhattan (and close areas) in April 2014!
len(data)

564516

In [7]:
# We can see that the Date/Time column is not in date time format
print(data['Date/Time'].dtype)

object


### Converting the Date/Time column in the correct format

In [8]:
data['Date/Time'] = pd.to_datetime(data['Date/Time'])
data['day_name'] = data['Date/Time'].dt.day_name()
data['hour'] = data['Date/Time'].dt.hour
data.head(10)

,Date/Time,Lat,Lon,Base,day_name,hour
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,Tuesday,0
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,Tuesday,0
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,Tuesday,0
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,Tuesday,0
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,Tuesday,0
5,2014-04-01 00:33:00,40.7383,-74.0403,B02512,Tuesday,0
6,2014-04-01 00:39:00,40.7223,-73.9887,B02512,Tuesday,0
7,2014-04-01 00:45:00,40.7620,-73.9790,B02512,Tuesday,0
8,2014-04-01 00:55:00,40.7524,-73.9960,B02512,Tuesday,0
9,2014-04-01 01:01:00,40.7575,-73.9846,B02512,Tuesday,1


## Time for some EDA !

### What are the busiest hours and the busiest days? 

In [9]:
def show_peak(data):

    '''This function will display peak hours and peak days using standard pd.DateTime terminology: day_name and hour '''

    # Preparing the plotting environnement
    plot = make_subplots(rows=1, cols=2, subplot_titles=('Count per hour', 'Count per day'))

    #### Count per day
    # Calculating count per day and mean + max
    day_count = pd.Series.value_counts(data['day_name'])
    
    day_count = pd.DataFrame(day_count).reset_index()
    day_mean = day_count['count'].mean()
    day_max = day_count['count'].max()

    # Reordering the dataframe rows according to the categorical weekdays
    std_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_count['day_name'] = pd.Categorical(day_count['day_name'], categories=std_order, ordered=True)
    day_count = day_count.sort_values('day_name').reset_index(drop=True)

    # Coloring the peak day and the days busier than average
    colors_d = []
    for count in day_count['count']:
        if count == day_max:
            colors_d.append('orange')
        elif count > day_mean:
            colors_d.append('grey')
        else:
            colors_d.append('lightgrey')

    # The actual count per day figure
    fig_d = go.Figure(go.Bar(x=day_count['day_name'], y=day_count['count'], marker=dict(color=colors_d)))
    fig_d.add_hline(y=day_mean, line_width=3, line_dash='solid', line_color='purple', row=1, col=2)   # To debug later: not showing!

    #### Count per hour
    # Calculating count per hour
    hour_count = pd.Series.value_counts(data['hour']).sort_index()

    # Coloring the peak hours based on if previous and next bars have inferior count or not
    colors_h = []
    for i in range(len(hour_count)):
        prev_count = hour_count[i - 1] if i > 0 else 0
        next_count = hour_count[i + 1] if i < len(hour_count) - 1 else 0
        if hour_count[i] == hour_count[0]:  # This is to avoid to account for the first bar, which will often appears as a peak because accounting prev_count as 0 and next_count is lower
            colors_h.append('lightgrey')
        elif hour_count[i] > prev_count and hour_count[i] > next_count:
            colors_h.append('orange') 
        else:
            colors_h.append('lightgrey')

    # The actual count per hour figure
    fig_h = go.Figure(go.Bar(x=hour_count.index, y=hour_count.values, marker=dict(color=colors_h)))

    # Remove legend from both histograms
    fig_h.update_traces(showlegend=False)
    fig_d.update_traces(showlegend=False)

    # Custom legends
    legend_entries = [
        dict(
            type='scatter',
            x=[None], y=[None],
            marker=dict(color='orange', size=15),
            mode='markers',
            showlegend=True,
            name='Peak Hours/Days'
        ),
        dict(
            type='scatter',
            x=[None], y=[None],
            marker=dict(color='grey', size=15),
            mode='markers',
            showlegend=True,
            name='Busier days than average'
        ),
        dict(
            type='scatter',
            x=[None], y=[None],
            marker=dict(color='lightgrey', size=15),
            mode='markers',
            showlegend=True,
            name='Regular Hours/Days'
        )
    ]

    # plot the two figures
    plot.add_trace(fig_h['data'][0], row=1, col=1)
    plot.add_trace(fig_d['data'][0], row=1, col=2)

   # Add custom legend traces
    for entry in legend_entries:
        plot.add_trace(entry)

    return plot

In [10]:
# applying the function on my dataframe
show_peak(data)

7am and 5pm are the busiests hours.

Wednesday is the busiest day. Tuesday, Thursday & Friday are busiest than average.

### Filter for the busiest hours

This will help reducing computing load during the testing phase, but we can re-integrate the whole dataset later!

In [11]:
mask_hour = (data['hour']==7)|(data['hour']==17)|(data['hour']==21)
data_trim = data[mask_hour]
print(f'Remaining hours in the dataset : {data_trim['hour'].unique()}')
print(f'Number of pickups remaining in the records: {len(data_trim)}')
print(f'We removed {len(data)-len(data_trim)} entries in total')

Remaining hours in the dataset : [ 7 17 21]
Number of pickups remaining in the records: 107363
We removed 457153 entries in total


## Machine Learning

I am choosing to train a DBscan. This is a density-based model of unsupervised Machine Learning, well suited for geographical segmentation

### This is a first test to calibrate the parameters

We only need the GPS coordinate to train the model, so I am dropping all the other columns for the test after filtering data on the peak day/hours

In [30]:
# I am choosing Wednesday to calibrate the DBscan parameters, as it is the busiest day
day_mask = (data_trim['day_name'] == 'Wednesday')&(data_trim['hour']==17)
wednesday = data_trim[day_mask]

# Cleaning the dataframe, focusing only on coordinate gps for training
X_peak = wednesday.drop(['Date/Time', 'day_name', 'hour', 'Base'], axis=1).reset_index(drop=True)

# This is the pipeline
numerical_features = X_peak.select_dtypes('float64').columns.tolist()
preprocessor = StandardScaler()

# Applying the pipeline
X_fit = preprocessor.fit_transform(X_peak)

# Fixing the parameters
# (eps = 0.071, min_samples = 35, metric = 'manhattan') -> very constrained
# (eps = 0.2, min_samples = 15, metric = 'manhattan') -> good baseline

db = DBSCAN(eps = 0.072, min_samples = 35, metric = 'manhattan')

In [31]:
# Training the model. CAUTION, this can take some time. See timer results.
start = time.time()
db.fit(X_fit)
end = time.time()
timer = end-start
print(f'The training took {timer:.1f} seconds')

The training took 0.0 seconds


In [32]:
# Preparing the output data for visualization
X_peak['clusters'] = db.labels_
outlier = X_peak['clusters']==-1  # DBscan create an outlier cluster, do not forget to isolate it
X_peak_out = X_peak[outlier]
X_peak = X_peak[~outlier]

# sorting the clusters by descending density
cluster_count = X_peak['clusters'].value_counts()
sorted_cluster = cluster_count.sort_values().index
cluster_mapping = {old: new for new, old in enumerate(sorted_cluster)}
X_peak['clusters'] = X_peak['clusters'].map(cluster_mapping)

print(f'dropped {outlier.sum()} outliers out of {len(wednesday)} values.')
print(X_peak['clusters'].value_counts())


dropped 2110 outliers out of 9151 values.
clusters
5    6556
4     235
3     104
2      54
1      48
0      44
Name: count, dtype: int64


In [71]:
## Plotting the clusters on map with noise in background

# This plots the noise as background
map_bgd = px.scatter_mapbox(X_peak_out, 'Lat', 'Lon', color_discrete_sequence=['lightgrey'], mapbox_style='carto-positron', zoom=9.5)

# This plots the clusters sorted according to density
map_clusters = px.scatter_mapbox(X_peak, 'Lat', 'Lon', color='clusters', mapbox_style='carto-positron', color_continuous_scale='BuGn', zoom=9.5)

# Updating layout so that the color scale from DB clusters is retained on the final map, and adding titles
map_bgd.update_layout(coloraxis=dict(colorscale='BuGn'),
                      title='Clusters of highest pickup concentrations. Avoid the grey zone!',
                      coloraxis_colorbar=dict(title='Clusters'))
for trace in map_clusters.data:
    map_bgd.add_trace(trace)

# The final map
map_bgd.show()


## Now that it is up and running, time to make it Usable on any day, any hour

The first step is to wrap everything in one simple-to-use function.

In [74]:
def show_clusters(data, day: str, hour, eps = 0.072, min_samples = 35):
    '''
    This function returns DBscan clusters of highest pickup concentrations on a map. It should be fed with data containing gps coordiantes, hour and day_names with
    correct pd.DateTimes terminology. DBscan hyperparameters (eps and min sample can be adjusted as well, default values were determined on peak hour (5pm) on the
    busiest day (wednesday)).
    '''
    # I am choosing Wednesday to calibrate the DBscan parameters, as it is the busiest day
    day_mask = (data['day_name'] == day)&(data['hour']==hour)
    data_day = data[day_mask]

    # Cleaning the dataframe, focusing only on coordinate gps for training
    X_peak = data_day.drop(['Date/Time', 'day_name', 'hour', 'Base'], axis=1).reset_index(drop=True)

    # This is the pipeline
    preprocessor = StandardScaler()

    # Applying the pipeline
    X_fit = preprocessor.fit_transform(X_peak)

    db = DBSCAN(eps = eps, min_samples = min_samples, metric = 'manhattan')

    # Training the model. CAUTION, this can take some time. See timer results.
    print('The model starts training.')
    start = time.time()
    db.fit(X_fit)
    end = time.time()
    timer = end-start
    print(f'The training took {timer:.1f} seconds')

    # Preparing the output data for visualization
    X_peak['clusters'] = db.labels_
    outlier = X_peak['clusters']==-1  # DBscan create an outlier cluster, do not forget to isolate it
    X_peak_out = X_peak[outlier]
    X_peak = X_peak[~outlier]

    # sorting the clusters by descending density
    cluster_count = X_peak['clusters'].value_counts()
    sorted_cluster = cluster_count.sort_values().index
    cluster_mapping = {old: new for new, old in enumerate(sorted_cluster)}
    X_peak['clusters'] = X_peak['clusters'].map(cluster_mapping)

    print(f'dropped {outlier.sum()} outliers out of {len(data_day)} values.')
    print(X_peak['clusters'].value_counts())    

    ## Plotting the clusters on map with noise in background

    # This plots the noise as background
    map_bgd = px.scatter_mapbox(X_peak_out, 'Lat', 'Lon', color_discrete_sequence=['lightgrey'], mapbox_style='carto-positron', zoom=9.5)

    # This plots the clusters sorted according to density
    map_clusters = px.scatter_mapbox(X_peak, 'Lat', 'Lon', color='clusters', mapbox_style='carto-positron', color_continuous_scale='BuGn', zoom=9.5)

    # Updating layout so that the color scale from DB clusters is retained on the final map, and adding titles
    map_bgd.update_layout(coloraxis=dict(colorscale='BuGn'),
                      title='Clusters of highest pickup concentrations. Avoid the grey zone!',
                      coloraxis_colorbar=dict(title='Clusters'))
    for trace in map_clusters.data:
        map_bgd.add_trace(trace)

    return map_bgd

## And now you can play with it, choose any day, any hour! 🚖

In [81]:
# As a reminder, to help you choose: 
show_peak(data)

In [87]:
# Default DBscan hyperparameters are based on previous test, but feel free to modify it if needed
show_clusters(data, day='Friday', hour=21)

The model starts training.
The training took 0.0 seconds
dropped 2324 outliers out of 6265 values.
clusters
4    3695
3      89
2      73
0      42
1      42
Name: count, dtype: int64
